# FIRE 

This notebook will try to answer the following questions:

- Given my current savings and no income, how many months will my savings last


## Assumptions

- The annual return on the stock investment (assumption ETF) is an average and is applied monthly. 
The assumption is a long term.
- The bond investment has a fixed return rate that's applied monthly. In reality that should be close to inflation rate.
- The monthly income is constant and is added to cash every month. It's also a simplification as in reality the income is not constant
- The monthly expenses are constant but the inflation rate is applied. In reality, the expenses won't be constant as they change depending on many factors.
- The inflation rate is constant
- When the cash is depleted, the investment is used to cover the expenses. The investment is used in the following order: cash, bond, stock, real estate. 
- Simulation ends when the wealth is depleted and no more assets are available to cover the expenses.


In [32]:
import sys
from pathlib import Path

project_root = Path.cwd().parent 
src_path = project_root / 'src'

sys.path.append(str(src_path))

In [33]:
import datetime
from fire.simulations import FireSimulation, run_simulation, InvestmentProperty
import pandas as pd
import plotly.express as px
from decimal import Decimal


In [34]:


init = FireSimulation(
    stock_investments=100_000,
    bonds_investments=130_000,
    cash=50_000,
    investment_properties=[
        InvestmentProperty(
            mortgage_left=Decimal("0"),
            mortgage_rate=Decimal("0"),
            mortgage_months=0,
            market_value=Decimal("800_000"),
            monthly_income=Decimal("2_300"),
        ),
        InvestmentProperty(
            mortgage_left=Decimal("292000"),
            mortgage_rate=Decimal("7.66"),
            mortgage_months=202,
            market_value=Decimal("520_000"),
            monthly_income=Decimal("2_100"),
        )
    ],
    return_rate_from_investment=0.08,
    return_rate_from_bonds=0.04,
    monthly_expenses=15_000,
    annual_inflation_rate=0.04,
    monthly_income=0,
    annual_property_appreciation_rate=Decimal('0.03'),
    invest_cash_surplus=True,
    invest_cash_threshold=50_000,
    invest_cash_surplus_strategy='80-20',
    start_date=datetime.datetime.fromisoformat('2024-03-01')
)

# simulate for next X years
years = 15
expected_number_of_months = years * 12
simulation = run_simulation(init, years*12)

In [35]:

df = pd.DataFrame([s.to_dict() for s in simulation])

number_of_records = len(df)
last_record = df.iloc[-1]


In [36]:
print(f"Without any income and burning only your investments, you can survive for {number_of_records} months")


Without any income and burning only your investments, you can survive for 104 months


In [37]:

fig = px.line(df, 
              x='start_date', 
              y=[
                  'cash', 
                  'monthly_expenses' ,
                  'stock_investments', 
                  'bonds_investments', 
                  ])

fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.25,
        xanchor="right",
        x=0.5
    )
)
fig

In [38]:
# I want to see how my wealth grows over time including a net property value

wealthFig = px.line(df,
                    x='start_date',
                    y=[
                        'properties_net_cash_value',
                        'wealth_inc_properties',
                        'liquid_wealth',
                    ])
wealthFig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.25,
        xanchor="right",
        x=0.5
    )
)

wealthFig.show()